Usando **R** no google Colab

In [ ]:
%load_ext rpy2.ipython

# Instalando e chamando pacotes

In [ ]:
%%R
install.packages("ggplot2")
library("ggplot2")

In [ ]:
%%R
help(package = "ggplot2")

In [ ]:
%%R
str(allPackage <- installed.packages())
allPackage [, c(1,3:5)]

# Primeiros passos

In [ ]:
%%R
altura <- 1.88
peso   <- 80
IMC    <- peso/(altura^2)
IMC

In [ ]:
%%R
ls()

In [ ]:
%%R
rm(list = ls())
ls()

# Criando vetores

In [ ]:
%%R
x <- c(2,5,7,5,7,2)
x

In [ ]:
%%R
y <- c(1,5,2,1,7,2)
xy <- x+y
xy

In [ ]:
%%R
xy * 2

In [ ]:
%%R
z <- c("Squall","Strife","Artorias","Ed")
z

In [ ]:
%%R
typeof(x)

In [ ]:
%%R
typeof(z)

In [ ]:
%%R
seq(10 , 100, by=10)

In [ ]:
%%R
rep(c(10,20,30) , 3)

In [ ]:
%%R
set.seed(1354)
runif(min = 1, max = 100, n = 10)

In [ ]:
%%R
set.seed(1354)
as.integer(runif(min = 1, max = 100, n = 10))

# Importando e manipulando bancos de dados

In [ ]:
%%R
dados <- read.csv("/content/AmostraBehavior.csv", sep = ";")
head(dados)

In [ ]:
%%R
dim(dados)

In [ ]:
%%R
dados = dados[sample(nrow(dados), 5000), ]
dim(dados)

In [ ]:
%%R
names(dados)

In [ ]:
%%R
dados$Renda_por_dependente <- dados$RendaMensal/dados$N_dependentes

In [ ]:
%%R
head(dados["Renda_por_dependente"])
head(dados$Renda_por_dependente)

In [ ]:
%%R
dados["Renda_por_dependente"] <- dados["RendaMensal"]/(dados["N_dependentes"]+1)

In [ ]:
%%R
dados_depend <- subset(dados, dados$N_dependentes > 0)
head(dados_depend)

In [ ]:
%%R
dados_depend2 <- subset(dados, dados$N_dependentes == 0 & dados$Idade > 40, select=c(Id, Perf_final, UltPercLimit, N_dependentes, Idade))
head(dados_depend2)

In [ ]:
%%R
dados_depend3 <- subset(dados, dados$N_dependentes %in% c(0,3) , select=c(Id, Perf_final, UltPercLimit, N_dependentes, Idade))
head(dados_depend3)

In [ ]:
%%R
dados_ord <- dados_depend2[order(dados_depend2$Idade),]
head(dados_ord,10)

# Agrupamentos e tabelas

In [ ]:
%%R
names(dados)

In [ ]:
%%R
tab <- table(dados$Perf_final)

In [ ]:
%%R
prop.table(tab)

Verificar se tem pegou mais emprestimos ter maior proporção de maus

In [ ]:
%%R
tab2 <- table(dados$Perf_final, dados$N_emprestimos)
prop.table(tab2,2)

In [ ]:
%%R
library("dplyr")

In [ ]:
%%R
sumar_dados <- dados %>% 
                        group_by(N_emprestimos) %>%
                        summarise(mean_renda  = mean(RendaMensal),
                                  mean_limite = mean(UltPercLimit),
                                  prop_mau    = mean(Perf_final))
sumar_dados

O perfil que mais atrasa usa mais o limite e pede mais emprestimos

# Gerando alguns gráficos

In [ ]:
%%R
counts <- table(dados$Perf_final)
barplot(counts, main = "Distribuição da performance", xlab = "Volume de bons e maus pagadores",
         names.arg=c("Bom", "Mau"), col = "#006699")

In [ ]:
%%R
dados$Perf_final2

dados$Perf_final2[dados$Perf_final == 1] <- "Mau"
dados$Perf_final2[dados$Perf_final == 0] <- "Bom"

boxplot(UltPercLimit ~ Perf_final2, data = dados, xlab = "Bons e maus pagadores",
   ylab =  "% de utilização do limite", main = "Boxplot do % de utilização do limite segundo a performance",
   col = "#006699")

In [ ]:
%%R
ggplot(data = dados, aes(x = Perf_final2, y = UltPercLimit, fill = Perf_final2, group = Perf_final2)) +
  geom_boxplot(aes(fill = Perf_final2),colour = "black") +
  geom_jitter(aes(color = Perf_final2), position = position_jitter(0.2), alpha = 0.1) +
  scale_fill_manual(values=c("#006699","#ff0000")) +
  xlab("Bons e maus pagadores") + ylab("% de utilização do limite") +
  scale_colour_manual(values=c("#006699","#ff0000")) +
  stat_summary(fun.y = median, geom = "point", shape = 23, size = 4, fill = "black", 
               color = "black") +
  facet_wrap(~ CAT_RendaMensal
  , scales = "free", ncol = 6) +
  theme(
        axis.ticks.x = element_blank(),
        strip.background = element_rect(fill="black"),
        strip.text.x = element_text(colour = "white", face="bold"),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        panel.background = element_blank(),
        plot.margin = margin(t = 0, r = 0, b = 0,l = 0, "cm")) -> boxgg

boxgg

dir1 <- "/content/box_gg.jpg"
ggsave(dir1, boxgg, device = "jpg", width = 28, height = 12, units = "cm", dpi = 120)